The networks found with Signor and Omnipath present some dicsonnected components (ax: HOXD9 and HOXD13).
Moreover, the network found with signor, introduced some complexes (C92, C6, C143...) that are missing the complementary gene/s.
In the following notebook, using the different strategies available in NeKo, we will try to complete the network using both SIGNOR and Omnipath.

In [1]:
from neko.core.network import Network
from neko._visual.visualize_network import NetworkVisualizer
from neko._inputs.resources import Resources
from neko._annotations.gene_ontology import Ontology
import omnipath as op
import pandas as pd
from neko._methods.compare import compare_networks
from yfiles_jupyter_graphs import GraphWidget

In [2]:
data_folder = "../../../data/"

figures_folder = "../../Figures/WNT/"

sif_output = "../../sif_files/WNT/"

In [ ]:
resources_signor = Resources()
resources_signor.import_signor_tsv(data_folder + "signor_db.tsv")

In [ ]:
omnipath = op.interactions.OmniPath().get()

In [ ]:
sif_signor = sif_output + "WNT_signor_Northcott.sif"
WNT_signor = Network(sif_file=sif_signor, resources=resources_signor.interactions)

In [ ]:
signor_initial_nodes = ['CTNNB1',
 'APC',
 'CSNK2B',
 'ARID1A',
 'ARID2',
 'ATM',
 'PIK3CA',
 'BAI3',
 'EPHA7',
 'DDX3X',
 'SYNCRIP',
 'ALX4',
 'DLX3',
 'DLX4',
 'EMX2',
 'EN1',
 'FOXF1',
 'HOMEZ',
 'HOXC4',
 'HOXC5',
 'HOXD11',
 'HOXD13',
 'HOXD8',
 'HOXD9',
 'IRX2',
 'IRX5',
 'LEF1',
 'LHX8',
 'MAF',
 'MSX2',
 'NKX6-1',
 'ONECUT2',
 'OSR2',
 'PAX3',
 'PITX1',
 'PRRX1',
 'RUNX2',
 'SHOX2']

In [ ]:
wnt_signor_visualizer = NetworkVisualizer(WNT_signor)
wnt_signor_visualizer.render(output_file="net_signor", view=True, highlight_nodes=signor_initial_nodes)

The following are the complex introduced by the NeKo from the Signor database

In [ ]:
for node in WNT_signor.nodes["Genesymbol"]:
    if node.startswith("SIGNOR"):
        print(node)

In [ ]:
signor_complexes = pd.read_csv("../../../data/SIGNOR_complexes.csv", sep=';')
signor_pf = pd.read_csv("../../../data/SIGNOR_PF.csv", sep=";")

In [ ]:
signor_pf

In [ ]:
signor_complexes

In [ ]:
for ID in signor_complexes["SIGNOR ID"]:
    if ID in WNT_signor.nodes["Genesymbol"].to_list():
        print(ID, ': ', signor_complexes.loc[signor_complexes["SIGNOR ID"] == ID]["LIST OF ENTITIES"].item(), signor_complexes.loc[signor_complexes["SIGNOR ID"] == ID]["COMPLEX NAME"].item())

In [ ]:
for ID in signor_pf["SIGNOR ID"]:
    if ID in WNT_signor.nodes["Genesymbol"].to_list():
        print(ID, ': ', signor_pf.loc[signor_pf["SIGNOR ID"] == ID]["PROT. FAMILY NAME"].item() + "_family")

In [ ]:
# Here I am adding all the nodes that are part of each complex in the network
for ID in signor_complexes["SIGNOR ID"]:
    if ID in WNT_signor.nodes["Genesymbol"].to_list():
        nodes = [item.strip() for item in signor_complexes.loc[signor_complexes["SIGNOR ID"] == ID]["LIST OF ENTITIES"].item().split(',')]
        for node in nodes: WNT_signor.add_node(node)

In [ ]:
# now I will use the function "connect_nodes" to search for direct connection between nodes (so paths with length 1)

WNT_signor.connect_nodes(only_signed=True, consensus_only=False)

In [ ]:
wnt_signor_visualizer = NetworkVisualizer(WNT_signor)
wnt_signor_visualizer.render(output_file="net_signor", view=True, highlight_nodes=signor_initial_nodes)

Signor uses its own way to name complexes and protein families. In order to make the network easier to read, we ae going to change name to those nodes automatically using a specific function in NeKo: 

In [ ]:
# Here I change the name of the complexes for better readability
for ID in signor_complexes["SIGNOR ID"]:
    if ID in WNT_signor.nodes["Genesymbol"].to_list():
        new_name = signor_complexes.loc[signor_complexes["SIGNOR ID"] == ID]["COMPLEX NAME"].item()
        WNT_signor.modify_node_name(old_name=ID, new_name=new_name, type="both")

In [ ]:
# Here I change the name of the protein families for better readability
for ID in signor_pf["SIGNOR ID"]:
    if ID in WNT_signor.nodes["Genesymbol"].to_list():
        new_name = signor_pf.loc[signor_pf["SIGNOR ID"] == ID]["PROT. FAMILY NAME"].item() + "_family"
        WNT_signor.modify_node_name(old_name=ID, new_name=new_name, type="both")

In [ ]:
# and here I remove the nodes that form complexes but that have no sources (basically useless input)
complexes_nodes = []
for ID in signor_complexes["COMPLEX NAME"]:
    if ID in WNT_signor.nodes["Genesymbol"].to_list():
        nodes = [item.strip() for item in signor_complexes.loc[signor_complexes["COMPLEX NAME"] == ID]["LIST OF ENTITIES"].item().split(',')]
        complexes_nodes.extend(nodes)
print(complexes_nodes)       
for node in WNT_signor.nodes["Uniprot"].unique():
    if node in complexes_nodes:
        if node not in WNT_signor.edges["target"].unique():
            WNT_signor.remove_node(node)

In [ ]:
wnt_signor_visualizer = NetworkVisualizer(WNT_signor)
wnt_signor_visualizer.render(output_file="net_signor", view=True, highlight_nodes=signor_initial_nodes)

In [ ]:
from neko._outputs.exports import Exports

In [ ]:
export = Exports(WNT_signor)

In [ ]:
export.export_sif(sif_output + "Signor_with_complexes_Northcott.sif")